# ML Model

## Import Libraries

In [ ]:
#sys libs
import os
import itertools
import shutil
import pathlib
from PIL import Image

#data handling tools
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
sns.set_style('whitegrid')
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report

#deep learning libs
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Conv2D , MaxPooling2D , Flatten , Activation , Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam , Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#IPYwidgets
import ipywidgets as widgets
from IPython.display import display
import io
from io import BytesIO

# Warnings
import warnings
warnings.filterwarnings("ignore")

## Data Preprocessing

## Read Data

In [ ]:
data_dir = '/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set'
filepaths = []
labels = []

folds = os.listdir(data_dir)

for fold in folds:
    foldpath = os.path.join(data_dir, fold)
    flist = os.listdir(foldpath)
    
    for f in flist:
        f_path = os.path.join(foldpath , f)
        filelist = os.listdir(f_path)
        
        for file in filelist:
            fpath = os.path.join(f_path , file)
            filepaths.append(fpath)
            
            if f == 'colon_aca':
                labels.append('Colon adenocarcinoma')
            
            elif f == 'colon_n':
                labels.append('Colon Benign Tissue')
                
            elif f == 'lung_aca':
                labels.append('Lung adenocarcinoma')
            
            elif f == 'lung_n':
                labels.append('Lung Benign Tissue')
                
            elif f == 'lung_scc':
                labels.append('Lung Squamous Cell Carcinoma')
                
Files = pd.Series(filepaths , name= 'filepaths')
labels = pd.Series(labels ,name = 'labels' )
df = pd.concat([Files , labels] , axis = 1)

In [ ]:
df

## Split data into train , validation and test

In [ ]:
#Split data into train 80%
train , dummy = train_test_split(df , train_size = 0.8 , shuffle = True , random_state= 40 , stratify= df['labels'])
#Split data into validation and test 10% both
valid , test = train_test_split(dummy , train_size = 0.5 , shuffle = True , random_state= 40 , stratify= dummy['labels'])

## Image Data Generator

In [ ]:
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

img_size = (224 , 224)

train_gen = tr_gen.flow_from_dataframe(train , x_col = 'filepaths' , y_col = 'labels' ,target_size = img_size ,
                                       class_mode= 'categorical',color_mode = 'rgb' , shuffle= True , batch_size= 16)

valid_gen = ts_gen.flow_from_dataframe(valid , x_col = 'filepaths' , y_col = 'labels' , target_size = img_size ,
                                      class_mode = 'categorical' , color_mode= 'rgb' , shuffle = True , batch_size= 16)
test_gen = ts_gen.flow_from_dataframe(test , x_col = 'filepaths' , y_col = 'labels' , target_size = img_size ,
                                     class_mode = 'categorical' , color_mode = 'rgb' , shuffle= False , batch_size = 16)

## Sample from train data

In [ ]:
dict_gen = train_gen.class_indices # defines dictionary {'class': index}
classes = list(dict_gen.keys()) #define list of dictionary that have the names of the classes in the train data
images , labels = next(train_gen) #get batch size sample from train generator 

plt.figure(figsize= (20,20))

for i in range(16):
    plt.subplot(4, 4, i+1)
    image = images[i] / 255 # scales data to range (0 - 255)
    plt.imshow(image)
    index = np.argmax(labels[i]) # get image index
    class_name = classes[index] # get class of image
    plt.title(class_name , color= 'blue' , fontsize = 12)
    plt.axis('off')
plt.show();

## Model Structure

In [ ]:
# Create Model Structure
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys())) # to define number of classes in dense layer

# create pre-trained model (you can built on pretrained model such as :  efficientnet, VGG , Resnet )
# we will use efficientnetv2b3 from EfficientNet family.
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(include_top= False, weights= "imagenet", input_shape= img_shape, pooling= 'max')
# base_model.trainable = False

model = Sequential([
    base_model,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
                bias_regularizer= regularizers.l1(0.006), activation= 'relu'),
    Dropout(rate= 0.45, seed= 123),
    Dense(class_count, activation= 'softmax')
])

model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

In [ ]:
keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)

## Iterate

In [ ]:
epochs = 10
history = model.fit(x= train_gen , epochs= epochs , validation_data= valid_gen , validation_steps=None , shuffle = False , verbose= 1)

## Model Performance

In [ ]:
# Define needed variables
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
index_loss = np.argmin(val_loss)
val_lowest = val_loss[index_loss]
index_acc = np.argmax(val_acc)
acc_highest = val_acc[index_acc]
Epochs = [i+1 for i in range(len(tr_acc))]
loss_label = f'best epoch= {str(index_loss + 1)}'
acc_label = f'best epoch= {str(index_acc + 1)}'

# Plot training history
plt.figure(figsize= (20, 8))
plt.style.use('fivethirtyeight')

plt.subplot(1, 2, 1)
plt.plot(Epochs, tr_loss, 'r', label= 'Training loss')
plt.plot(Epochs, val_loss, 'g', label= 'Validation loss')
plt.scatter(index_loss + 1, val_lowest, s= 150, c= 'blue', label= loss_label)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(Epochs, tr_acc, 'r', label= 'Training Accuracy')
plt.plot(Epochs, val_acc, 'g', label= 'Validation Accuracy')
plt.scatter(index_acc + 1 , acc_highest, s= 150, c= 'blue', label= acc_label)
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout
plt.show()

## Model Evaluation

In [ ]:
ts_length = len(test)
test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size

train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Valid Loss: ", valid_score[0])
print("Valid Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

## Model Prediction

In [ ]:
preds = model.predict_generator(test_gen)
y_pred = np.argmax(preds , axis = 1)

## Confusion Matrix and Classification Report

In [ ]:
g_dict = test_gen.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test_gen.classes, y_pred)

plt.figure(figsize= (10, 10))
plt.imshow(cm, interpolation= 'nearest', cmap= plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation= 45)
plt.yticks(tick_marks, classes)


thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment= 'center', color= 'white' if cm[i, j] > thresh else 'black')

plt.tight_layout()
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

plt.show()

In [ ]:
# Classification report
print(classification_report(test_gen.classes, y_pred, target_names= classes))

In [ ]:
#Save the model
model.save('Model.h5')

## Prediction using loaded_model

In [ ]:
loaded_model = tf.keras.models.load_model('/kaggle/working/Model.h5', compile=False)
loaded_model.compile(Adam(learning_rate= 0.0001) , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [ ]:
# Create the widget
imageFile = widgets.FileUpload(
    accept='.jpeg',   # The type of files to accept
    multiple=False    # Whether to allow to upload multiple files or not
)

# Display the widget
display(imageFile)

In [ ]:
for name, file_info in imageFile.value.items():
    image = Image.open(io.BytesIO(file_info['content']))

# Preprocess the image
img = image.resize((224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# Make predictions
predictions = loaded_model.predict(img_array)
classes = list(dict_gen.keys())
class_labels = classes
score = tf.nn.softmax(predictions[0])
output = class_labels[tf.argmax(score)]
print(f"{class_labels[tf.argmax(score)]}")

In [ ]:
if output == "Colon adenocarcinoma":
    intent_filename = "colon_aca_intents"
elif output == "Lung adenocarcinoma":
    intent_filename = "lung_aca_intents"
elif output == "Lung Squamous Cell Carcinoma":
    intent_filename = "lung_scc_intents"
else:
    intent_filename = "no_cancer_intents"

# NLP Model

In [ ]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
filepath = f"/kaggle/input/intents/{intent_filename}.json"
data_file = open(filepath).read()
intents = json.loads(data_file)

In [ ]:
# preprocessing the json data
# tokenization
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# lemmatize, lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))

In [ ]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training, dtype="object")

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

In [ ]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

In [ ]:
# load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open(filepath).read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [ ]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
# function to start the chat bot which will continue till the user type 'end'

def start_chat():
    print("Bot: Your Personal Assistant is ready.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [ ]:
# start the chat bot
start_chat()